In [1]:
import os

In [2]:
# Specify the code source directory 
parent_directory = 'c:/YieldPredictionfromImagery/src' # change to your path

# set the default path for the code
os.chdir(parent_directory) 
# %pwd

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
# import csv

Import functions from other modules

In [4]:
# from dataset import select_data_and_yield_list
from dl.dl_prediction import select_imglist_yield
from ml.ml_predict import ml_predict_yield, prepare_train_test_data

In [5]:
from config_constant import crop_var, irrigate_var, CROP_TYPE
from plot_utils import plot_distinct_yields

Specify the path of labeled yield file and weather data

In [6]:
# 
yield_file = 'C:/Corn_Yield/BL2022_Yld_label.csv'
weather_file = 'C:/Corn_Yield/CoAgMet ET_VBA_calc_2022.xlsm'
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220720_DOY201_extracted_filled'
# out_path = '../output/DOY201/'
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220926_DOY269_extracted_filled'
# out_path = '../output/DOY269/'
# doy=269
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220916_DOY259_extracted_filled'
# out_path = '../output/DOY259/'
# doy = 259
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220716_DOY197_extracted_filled'
# out_path = '../output/DOY197/'
# doy = 197
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220708_DOY189_extracted_filled'
# out_path = '../output/DOY189/'
img_path = 'C:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220705_DOY186_extracted_filled'
out_path = '../output/DOY186/'
doy = 186
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220628_DOY179_extracted_filled'
# out_path = '../output/DOY179/'
# doy=179
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220616_DOY167_extracted_filled'
# out_path = '../output/DOY167/'
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220610_DOY161_extracted_filled'
# out_path = '../output/DOY161/'
if not os.path.exists(out_path):
    # If it doesn't exist, create the directory and any missing parent directories
    os.makedirs(out_path)
    print(f"Directory '{out_path}' created successfully.")
else:
    print(f"Directory '{out_path}' already exists.")

name_tag = img_path[-23:-17]

Directory '../output/DOY186/' already exists.


In [7]:
suf_name ={'LWIR': 'Thermal'}

Predict corn yield from drone Image. 

In [8]:
''''''
key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']

# key_word_list = ['Ref_filled.tif']
# suffix_list_list = [[], ['LWIR_filled.tif']]
# key_word_list = ['RGB_filled2.tif']
suffix_list_list = [[]]
# suffix = ['base', 'lwir']
# suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi',  'evi', 'ndre', 'gndvi']
VI_list = []
vi_only = False

analyze_variety_id=2
analyze_irrigation_id=2
train_col='TRAIN_75'
'''keyword to specify use rgb image or 5-bands image'''
for keyword in key_word_list:    
    
    '''get matching yield and image data'''
    img_list, yield_pf = select_imglist_yield(yield_file, img_path, keyword, 
                                            analyze_variety_id=analyze_variety_id, analyze_irrigation_id=analyze_irrigation_id)
    
    '''split the data into training and test part'''
    for suffix_list in suffix_list_list:
        train_images, test_images, train_yields, test_yields= prepare_train_test_data(img_list, yield_pf,train_col=train_col, 
                            VI_list=VI_list, suffix_list=suffix_list, vi_only=False)
        
        '''specify the machine learning modes in the 'test_modes', which will be passed to the function 'ml_predict_yield'
        the parameters of ml algorithm can be changed inside the 'ml_predict_yield' function
        '''
        test_modes = ['Lasso', 'LR']
        # test_modes = ['Lasso', 'LR', 'GB', 'RF', 'SVR','XGB']
        # test_modes = ['Lasso', 'LR', 'RF']
        # test_modes = ['Lasso',  'RF', 'GB']
        for modelname in test_modes:
            
            out_name = name_tag + ' ' + CROP_TYPE[crop_var[analyze_variety_id]] + ' ' 
            if not vi_only:
                out_name = out_name + keyword[:3]
            for vi_name in VI_list:
                out_name = out_name + '+' + vi_name.upper()
            for suf in suffix_list:
                out_name = out_name + '+' + suf_name[suf[:suf.rfind('_')]]
            out_name = out_name + ' '
            
            cur_time = time.time()
            trained_model = ml_predict_yield(train_images, train_yields, test_images, modelname, out_name, out_path)
            
            training_time = time.time() - cur_time
            print('training time is : ', training_time)
            
            
            cur_time = time.time()
            pred_yields = trained_model.predict(test_images)
            predict_time = time.time() - cur_time
            print('predict time is : ', predict_time)
            
            test_irrigate_data = np.array(yield_pf[yield_pf[train_col] == 0]['Irrigation_int'])
            test_variety_data = np.array(yield_pf[yield_pf[train_col] == 0]['Variety_int'])

            title = out_name +'('+ modelname +') '
            out_name = out_name + modelname +' '


            result_df=pd.DataFrame({
                'Truth': np.array(test_yields), 
                'Prediction':np.array(pred_yields),
                'Irrigation_int':test_irrigate_data,
                'Vriaty_int':test_variety_data
            })
            csv_file_path = out_path+out_name + '.csv'
            result_df.to_csv(csv_file_path, index=False)
            
            
            test_yields = np.array(test_yields)
            pred_yields = np.array(pred_yields)
            '''plot the results'''
            plot_distinct_yields(np.array(test_yields)*0.06277, np.array(pred_yields)*0.06277, test_irrigate_data, test_variety_data, title, out_path+out_name)
                        
#           
            if analyze_variety_id >2:
                for var_id in set(test_variety_data):
                    
                    subtitle = title + str(var_id)
                    sub_test_yields = test_yields[test_variety_data == var_id]
                    sub_pred_yields = pred_yields[test_variety_data == var_id]
                    sub_test_irrigate_data = test_irrigate_data[test_variety_data == var_id]
                    sub_test_variety_data = test_variety_data[test_variety_data == var_id]
                    plot_distinct_yields(sub_test_yields, sub_pred_yields, 
                                            sub_test_irrigate_data, sub_test_variety_data, subtitle, out_path+subtitle)
    #a

training time is :  0.39975690841674805
predict time is :  0.03338265419006348
../output/DOY186/DOY186 Pioneer Ref Lasso   r-squared = 0.22796391125103355, rmse = 2.649228104595925, mae=2.394182383783855 20.433008623144442
training time is :  0.4001893997192383
predict time is :  0.03344011306762695
../output/DOY186/DOY186 Pioneer Ref LR   r-squared = 0.7491178008723696, rmse = 1.5102035146579864, mae=1.192006006063432 10.173105092453532
training time is :  0.21663737297058105
predict time is :  0.01649308204650879
../output/DOY186/DOY186 Pioneer RGB Lasso   r-squared = 0.6929541007773863, rmse = 1.6707143524327703, mae=1.2925393094310862 11.031100652247913
training time is :  0.20010638236999512
predict time is :  0.019542455673217773
../output/DOY186/DOY186 Pioneer RGB LR   r-squared = 0.6931081344170611, rmse = 1.6702952318768611, mae=1.2916421946464405 11.023444278926158


# Gridsearch

In [ ]:
# param_grid = {
#     'n_estimators': [100, 150, 200],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }
# MEAN = np.nanmean(train_images,(0,2,3))
# STD = np.nanstd(train_images, (0,2,3))
# train_images = (train_images- MEAN[None,:,None,None])/STD[None,:,None,None] 
# train_images = train_images.reshape((train_images.shape[0], -1))
# rf_model = RandomForestRegressor(random_state=42)

# # Create GridSearchCV instance with the model, parameter grid, and cross-validation
# grid_search = GridSearchCV(estimator=rf_model
# , param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# # Perform the grid search on the dataset
# grid_search.fit(train_images, train_yields)

# # Print the best parameters and corresponding mean squared error
# print("Best Parameters:", grid_search.best_params_)
# print("Best Mean Squared Error:", -grid_search.best_score_)

In [ ]:
# # Initiaze the hyperparameters for each dictionary
# param1 = {}
# param1['classifier'] = [LinearRegression()]

# param2 = {}
# param2['classifier__alpha'] = [0.1, 0.3, 0.5, 1]
# param2['classifier'] = [Ridge()]

# param3 = {}
# param3['classifier__alpha'] = [0.2, 0.3, 0.4, 0.5, 0.6]
# param3['classifier'] = [Lasso()]

# param4 = {}
# param4['classifier__n_neighbors'] = [2,5,10,25,50]
# param4['classifier'] = [KNeighborsRegressor()]

# """
# param5 = {}
# param5['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
# param5['classifier__epsilon'] = [0.0,0.2,0.5,1]
# param5['classifier'] = [LinearSVR()]
# """

In [ ]:
# pipeline = pipe = Pipeline(steps=[("scaler", CustomScaler(MEAN,STD)),
#                        ("flatten", FlattenTransformer()),
#                        ("classifier", LinearRegression())])
# params = [param1, param2, param3, param4] # param5
# params = [param1, param2, param3, param4] # param5


In [ ]:
# # Train the grid search model
# grid_search = GridSearchCV(pipeline, params, cv=3, scoring='neg_mean_squared_error').fit(train_images,train_yields)

In [ ]:
# # Best performing model and its corresponding hyperparameters
# grid_search.best_params_

In [ ]:
# model = grid_search.best_estimator_
# # predict
# pred = model.predict(train_images)

# # validate mse error
# mse = mean_squared_error(train_yields, pred) 
# rmse = np.sqrt(mse)

# print(mse, rmse)

In [ ]:
# # model = grid_search.best_estimator_
# # predict
# pred = model.predict(test_images)

# # validate mse error
# mse = mean_squared_error(test_yields, pred) 
# rmse = np.sqrt(mse)

# print(mse, rmse)

In [ ]:
# save_name = 'test'
# plot_result(test_yields, pred, save_name)